# Prerequisites


In [2]:
# pip install GTSAM to use the pose in SE(2)
!pip install gtsam

     |████████████████████████████████| 7.8MB 2.8MB/s 


In [0]:
# pylint: disable=invalid-name, E1101

from __future__ import print_function

import math
import unittest
from functools import reduce

import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D  # pylint: disable=W0611

import matplotlib.patches as mpatches
import matplotlib.transforms as mtransforms
import mpl_toolkits.mplot3d.axes3d as p3

# Required to do animations in colab
from matplotlib import animation
from IPython.display import HTML

import gtsam
import gtsam.utils.plot as gtsam_plot
from gtsam import Pose2

In [0]:
# Set plot parameters
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (8.0, 8.0)
plt.rc('animation', html='jshtml') # needed for animations!
arrowOptions = dict(head_width=.02,head_length=.02, width=0.01)

# Utility Funcions
You can use the following functions to compute the different vector operations that you might need during the assignemnt.

In [0]:
# Some utility functions for Pose2
def vector3(x, y, z):
    """Create 3D double numpy array."""
    return np.array([x, y, z], dtype=np.float)


def compose(*poses):
    """Compose all Pose2 transforms given as arguments from left to right."""
    return reduce((lambda x, y: x.compose(y)), poses)


def vee(M):
    """Pose2 vee operator."""
    return vector3(M[0, 2], M[1, 2], M[1, 0])


def delta(g0, g1):
    """Difference between x,y,,theta components of SE(2) poses."""
    return vector3(g1.x() - g0.x(), g1.y() - g0.y(), g1.theta() - g0.theta())


def trajectory(g0, g1, N=20):
    """ Create an interpolated trajectory in SE(2), treating x,y, and theta separately.
        g0 and g1 are the initial and final pose, respectively.
        N is the number of *intervals*
        Returns N+1 poses
    """
    e = delta(g0, g1)
    return [Pose2(g0.x()+e[0]*t, g0.y()+e[1]*t, g0.theta()+e[2]*t) for t in np.linspace(0, 1, N)]

# Manipulator Class
We will work with a three link arm with three revolute joints (RRR). This class has its initialization function that should be left as it is and you will have to work if the other functions to have a working system.

The functions that you will have to implement are:


*   fk: Forward Kinematics
*   con: Solving Forward Kinematics using conjugation form
*   poe: Solving Forward Kinematics using power of exponentials form
*   Jacobian: Computes the manipulator Jacobian in SE(2)
*   ik: Computes Inverse Kinematics



In [0]:
# The 3-link manipulator class

class ThreeLinkArm(object):
    """Three-link arm class."""

    def __init__(self):
        self.L1 = 3.5
        self.L2 = 3.5
        self.L3 = 2.5

    def fk(self, q):
        """ Forward kinematics.
            Takes numpy array of joint angles, in radians.
        """
        return Pose2()

        
    def con(self, q):
        """ Forward kinematics, conjugation form.
            Takes numpy array of joint angles, in radians.
        """
        return Pose2()

    def poe(self, q):
        """ Forward kinematics.
            Takes numpy array of joint angles, in radians.
        """
        return Pose2()

    def jacobian(self, q):
        """ Calculate manipulator Jacobian.
            Takes numpy array of joint angles, in radians.
        """
        return np.eye(3)

    def ik(self, sTt_desired, e=1e-9):
        """ Inverse kinematics.
            Takes desired Pose2 of tool T with respect to base S.
            Optional: e: error norm threshold
        """
        q = np.radians(vector3(30, -30, 45))  # take initial estimate well within workspace

        # return result in interval [-pi,pi)
        return np.remainder(q+math.pi, 2*math.pi)-math.pi



# UnitTest
Your assignment will be graded as you pass the following unitTest. 
For now, we are skipping the 5 unit tests. The assignment consists in fixing them all, and then using the result to do trajectory following for the robot.

In [7]:

# Create common example configurations.
Q0 = vector3(0, 0, 0)
Q1 = np.radians(vector3(-30, -45, -90))
Q2 = np.radians(vector3(-90, 90, 0))

class TestPose2SLAMExample(unittest.TestCase):
    """Unit tests for functions used below."""

    def setUp(self):
        self.arm = ThreeLinkArm()

    def assertPose2Equals(self, actual, expected, tol=1e-2):
        """Helper function that prints out actual and expected if not equal."""
        equal = actual.equals(expected, tol)
        if not equal:
            raise self.failureException(
                "Poses are not equal:\n{}!={}".format(actual, expected))

    @unittest.skip("Skipping FK")
    def test_fk(self):
        """Make sure forward kinematics is correct for some known test configurations."""
        # at rest
        expected = Pose2(0, 2*3.5 + 2.5, math.radians(90))
        sTt = self.arm.fk(Q0)
        self.assertIsInstance(sTt, Pose2)
        self.assertPose2Equals(sTt, expected)

        # -30, -45, -90
        expected = Pose2(5.78, 1.52, math.radians(-75))
        sTt = self.arm.fk(Q1)
        self.assertPose2Equals(sTt, expected)

    @unittest.skip("Skipping CON")
    def test_con(self):
        """Make sure POE is correct for some known test configurations."""
        # at rest
        expected = Pose2(0, 2*3.5 + 2.5, math.radians(90))
        sTt = self.arm.con(Q0)
        self.assertIsInstance(sTt, Pose2)
        self.assertPose2Equals(sTt, expected)

        # -30, -45, -90
        expected = Pose2(5.78, 1.52, math.radians(-75))
        sTt = self.arm.con(Q1)
        self.assertPose2Equals(sTt, expected)

    @unittest.skip("Skipping POE")
    def test_poe(self):
        """Make sure POE is correct for some known test configurations."""
        # at rest
        expected = Pose2(0, 2*3.5 + 2.5, math.radians(90))
        sTt = self.arm.poe(Q0)
        self.assertIsInstance(sTt, Pose2)
        self.assertPose2Equals(sTt, expected)

        # -30, -45, -90
        expected = Pose2(5.78, 1.52, math.radians(-75))
        sTt = self.arm.poe(Q1)
        self.assertPose2Equals(sTt, expected)

    @unittest.skip("Skipping Jacobian")
    def test_jacobian(self):
        """Test Jacobian calculation."""
        # at rest
        expected = np.array([[-9.5, -6, -2.5], [0, 0, 0], [1, 1, 1]], np.float)
        J = self.arm.jacobian(Q0)
        np.testing.assert_array_almost_equal(J, expected)

        # at -90, 90, 0
        expected = np.array([[-6, -6, -2.5], [3.5, 0, 0], [1, 1, 1]], np.float)
        J = self.arm.jacobian(Q2)
        np.testing.assert_array_almost_equal(J, expected)

    @unittest.skip("Skipping IK")
    def test_ik(self):
        """Check iterative inverse kinematics function."""
        # at rest
        actual = self.arm.ik(Pose2(0, 2*3.5 + 2.5, math.radians(90)))
        np.testing.assert_array_almost_equal(actual, Q0, decimal=2)

        # -30, -45, -90
        sTt_desired = Pose2(5.78, 1.52, math.radians(-75))
        actual = self.arm.ik(sTt_desired)
        self.assertPose2Equals(self.arm.fk(actual), sTt_desired)
        np.testing.assert_array_almost_equal(actual, Q1, decimal=2)

if __name__ == '__main__':
  unittest.main(argv=['first-arg-is-ignored'], exit=False)

sssss
----------------------------------------------------------------------
Ran 5 tests in 0.005s

OK (skipped=5)


# Plot Example

You can run this example to see what the robot is doing



In [8]:
# First set up the figure, the axis, and the plot element we want to animate
fig, ax = plt.subplots()
plt.close()
N=50
size=10.5
ax.set_xlim((-size, size))
ax.set_ylim((-size, size))
omega = 2*math.pi/N

arm = ThreeLinkArm()
q = np.radians(vector3(30, -30, 45))
sTt_initial = arm.fk(q)
sTt_goal = Pose2(2.4, 4.3, math.radians(0))
poses = trajectory(sTt_initial, sTt_goal, N)

def init():
  rect = mpatches.Rectangle([0,0], 1, 1, angle =0)
  return (rect,)

# animation function. This is called sequentially  
def animate(i):
  global pose
  global arm
  global q

  # Computes the forward kinematics to get the pose of the end-effector for the given angular position of the joints (q)
  sTt = arm.fk(q)
  # Evaluate the error between the current position of the end-effector and the desired position at moment i
  error = delta(sTt, poses[i])
  # Get the jacobian of the arm at the given pose
  J = arm.jacobian(q)
  # Move the arm joints in the respective direction
  q += np.dot(np.linalg.inv(J), error)

  # ------------------------- ANIMATION ----------------------------------------------------
  rect = rect = mpatches.Rectangle([-0.5,-0.5], 1, 1, angle =0)
  ax.clear()
  ax.set_xlim((-size, size))
  ax.set_ylim((-size, size))
  ax.add_artist(rect)
  
  sXl1 = Pose2(0, 0, math.radians(90))
  l1Zl1 = Pose2(0, 0, q[0])
  l1Xl2 = Pose2(arm.L1, 0, 0)
  sTl2 = compose(sXl1, l1Zl1, l1Xl2)
  t1 = sTl2.translation()
  ax.add_artist(mpatches.Rectangle([0,0], 3.5, 0.1, angle =q[0]*180/np.pi+90, color='r'))

  l2Zl2 = Pose2(0, 0, q[1])
  l2Xl3 = Pose2(arm.L2, 0, 0)
  sTl3 = compose(sTl2, l2Zl2, l2Xl3)
  t2 = sTl3.translation()
  ax.add_artist(mpatches.Rectangle([t1.x(),t1.y()], 3.5, 0.1, angle =(q[0]+q[1])*180/np.pi+90, color='g'))

  l3Zl3 = Pose2(0, 0, q[2])
  l3Xt = Pose2(arm.L3, 0, 0)
  sTt = compose(sTl3, l3Zl3, l3Xt)
  t3 = sTt.translation()
  ax.add_artist(mpatches.Rectangle([t2.x(),t2.y()], 2.5, 0.1, angle =(q[0]+q[1]+q[2])*180/np.pi+90, color='b'))

animation.FuncAnimation(fig, animate, init_func=init, 
                        frames=N, interval=100, blit=False)

# Bonus Question
Know the base is moving with a linear velocity of [0.1 0.05] meters per iteration (at each time step, the base has advance that distance in x and y). We consider that the base can move in this direction without rotating. 

For the bonus points, change the following code, such that, the arm still goes to the goal pose (2.4, 4.3, 0), regardless of the motion of the base.

In [0]:
# First set up the figure, the axis, and the plot element we want to animate
fig, ax = plt.subplots()
plt.close()
N=50
size=10.5
ax.set_xlim((-size, size))
ax.set_ylim((-size, size))
omega = 2*math.pi/N

arm = ThreeLinkArm()
q = np.radians(vector3(30, -30, 45))
sTt_initial = arm.fk(q)
sTt_goal = Pose2(2.4, 4.3, math.radians(0))
poses = trajectory(sTt_initial, sTt_goal, N)

def init():
  rect = mpatches.Rectangle([0,0], 1, 1, angle =0)
  return (rect,)

# animation function. This is called sequentially  
def animate(i):
  global pose
  global arm
  global q

  # Computes the forward kinematics to get the pose of the end-effector for the given angular position of the joints (q)
  Base_Pose = Pose2(i*0.1, i*0.05, 0)
  sTt = arm.fk(q)
  sTt = Pose2(sTt.x(), sTt.y(), sTt.theta())
  # Evaluate the error between the current position of the end-effector and the desired position at moment i
  error = delta(sTt, poses[i])
  # Get the jacobian of the arm at the given pose
  J = arm.jacobian(q)
  # Move the arm joints in the respective direction
  q += np.dot(np.linalg.inv(J), error)

  # ------------------------- ANIMATION ----------------------------------------------------
  rect = rect = mpatches.Rectangle([-0.5+Base_Pose.x(),-0.5+Base_Pose.y()], 1, 1, angle =0)
  ax.clear()
  ax.set_xlim((-size, size))
  ax.set_ylim((-size, size))
  ax.add_artist(rect)
  
  sXl1 = Pose2(0, 0, math.radians(90))
  l1Zl1 = Pose2(0, 0, q[0])
  l1Xl2 = Pose2(arm.L1, 0, 0)
  sTl2 = compose(sXl1, l1Zl1, l1Xl2)
  t1 = sTl2.translation()
  ax.add_artist(mpatches.Rectangle([Base_Pose.x(),Base_Pose.y()], 3.5, 0.1, angle =q[0]*180/np.pi+90, color='r'))

  l2Zl2 = Pose2(0, 0, q[1])
  l2Xl3 = Pose2(arm.L2, 0, 0)
  sTl3 = compose(sTl2, l2Zl2, l2Xl3)
  t2 = sTl3.translation()
  ax.add_artist(mpatches.Rectangle([t1.x()+Base_Pose.x(),t1.y()+Base_Pose.y()], 3.5, 0.1, angle =(q[0]+q[1])*180/np.pi+90, color='g'))

  l3Zl3 = Pose2(0, 0, q[2])
  l3Xt = Pose2(arm.L3, 0, 0)
  sTt = compose(sTl3, l3Zl3, l3Xt)
  t3 = sTt.translation()
  ax.add_artist(mpatches.Rectangle([t2.x()+Base_Pose.x(),t2.y()+Base_Pose.y()], 2.5, 0.1, angle =(q[0]+q[1]+q[2])*180/np.pi+90, color='b'))

animation.FuncAnimation(fig, animate, init_func=init, 
                        frames=N, interval=100, blit=False)